In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Funções auxiliares
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

def train_mlp(X_train, y_train, X_test, y_test, hidden_units=10, learning_rate=0.01, epochs=10000):
    # Inicialização dos pesos e bias
    input_layer_size = X_train.shape[1]
    output_layer_size = 1
    np.random.seed(42)

    # Pesos e bias para a camada de entrada para camada oculta
    W1 = np.random.randn(input_layer_size, hidden_units)
    b1 = np.zeros((1, hidden_units))

    # Pesos e bias para a camada oculta para camada de saída
    W2 = np.random.randn(hidden_units, output_layer_size)
    b2 = np.zeros((1, output_layer_size))

    # Lista para salvar os erros durante o treinamento
    errors = []

    # Treinamento
    for epoch in range(epochs):
        # Feedforward - camadas ocultas
        Z1 = np.dot(X_train, W1) + b1
        A1 = sigmoid(Z1)

        # Camada de saída
        Z2 = np.dot(A1, W2) + b2
        A2 = sigmoid(Z2)

        # Cálculo do erro
        error = y_train.reshape(-1, 1) - A2
        errors.append(np.mean(np.abs(error)))

        # Backpropagation
        dA2 = error * sigmoid_derivative(A2)
        dW2 = np.dot(A1.T, dA2)
        db2 = np.sum(dA2, axis=0, keepdims=True)

        dA1 = np.dot(dA2, W2.T) * sigmoid_derivative(A1)
        dW1 = np.dot(X_train.T, dA1)
        db1 = np.sum(dA1, axis=0, keepdims=True)

        # Atualização dos pesos e bias
        W1 += learning_rate * dW1
        b1 += learning_rate * db1
        W2 += learning_rate * dW2
        b2 += learning_rate * db2

        # Printar erro a cada 1000 iterações
        if epoch % 1000 == 0:
            print(f"Epoch {epoch}/{epochs}, Error: {np.mean(np.abs(error))}")

    # Predição nos dados de teste
    Z1 = np.dot(X_test, W1) + b1
    A1 = sigmoid(Z1)
    Z2 = np.dot(A1, W2) + b2
    A2 = sigmoid(Z2)

    predictions = (A2 > 0.5).astype(int).flatten()
    return predictions, errors

# Carregar e pré-processar os dados
dataset2_inputs = pd.read_csv('data/dataset2_inputs.csv', sep='\t')
test_in = pd.read_csv('data/test_in.csv', sep='\t')
test_out = pd.read_csv('data/teste_out.csv', sep='\t')

# Vetorização do texto usando TF-IDF
vectorizer = TfidfVectorizer(max_features=5000)
X_train = vectorizer.fit_transform(test_in['Text']).toarray()

# Codificação dos rótulos (AI ou Human)
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(test_out['Label'])

# Divisão dos dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Treinar o modelo MLP manual
predictions, _ = train_mlp(X_train, y_train, X_test, y_test, hidden_units=20, learning_rate=0.01, epochs=10000)

# Verificar o tamanho das previsões e do dataset2_inputs
print(f"Tamanho do dataset2_inputs: {dataset2_inputs.shape[0]}")
print(f"Tamanho das previsões: {len(predictions)}")

# Ajustar as previsões para garantir que a quantidade de previsões seja a mesma que o número de linhas em dataset2_inputs
if len(predictions) != dataset2_inputs.shape[0]:
    # Se o número de previsões não coincidir, ajustar as previsões
    predictions = predictions[:dataset2_inputs.shape[0]]

# Agora podemos adicionar as previsões corretamente ao DataFrame
predictions_label = label_encoder.inverse_transform(predictions)
dataset2_inputs['Predictions'] = predictions_label

# Mostrar as previsões no dataset2_inputs e salvar o arquivo CSV
dataset2_inputs[['ID', 'Predictions']].to_csv('data/dataset2_predictions_mlp-s1.csv', index=False)

print("Previsões geradas e salvas em 'data/dataset2_predictions_mlp.csv'")


Epoch 0/10000, Error: 0.49638004597796964
Epoch 1000/10000, Error: 0.017543229847077676
Epoch 2000/10000, Error: 0.009791453910807067
Epoch 3000/10000, Error: 0.0073402322133975404
Epoch 4000/10000, Error: 0.006058067944126904
Epoch 5000/10000, Error: 0.005247184159645864
Epoch 6000/10000, Error: 0.004678638321979209
Epoch 7000/10000, Error: 0.00425318025952555
Epoch 8000/10000, Error: 0.003920166515513606
Epoch 9000/10000, Error: 0.0036507951632207053
Tamanho do dataset2_inputs: 100
Tamanho das previsões: 200
Previsões geradas e salvas em 'data/dataset2_predictions_mlp.csv'
